In [3]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import numpy
import csv
#from hdbscan import HDBSCAN
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from cuml.preprocessing import normalize

In [ ]:
df = pd.read_csv('../data/Data_out.csv')

In [5]:
docs = df['clean_docs'].to_list()

In [6]:
with numpy.load('/kaggle/input/yt-embebbings/embeddings.csv.npz') as data:
    embeddings = data['arr_0']

In [7]:
embeddings = normalize(embeddings)


/opt/conda/lib/python3.10/site-packages/cupy/cuda/compiler.py:233: PerformanceWarning: Jitify is performing a one-time only warm-up to populate the persistent cache, this may take a few seconds and will be improved in a future release...
  jitify._init_module()


In [8]:
umap_model =UMAP(
            n_neighbors=15,
            n_components=5,
            min_dist=0.0,
            metric="cosine",
        )

In [9]:
vectorizer_model = CountVectorizer(stop_words="english")


In [10]:
hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=5)
topic_model = BERTopic(vectorizer_model=vectorizer_model,low_memory=True,hdbscan_model=hdbscan_model,umap_model=umap_model)
topics, probs = topic_model.fit_transform(docs,embeddings)
df = topic_model.get_document_info(docs)

[I] [05:03:32.442584] Transform can only be run with brute force. Using brute force.


In [11]:
#reduced_embeddings = UMAP(n_neighbors=10, n_components=2, 
#                          min_dist=0.0, metric='cosine').fit_transform(embeddings[:100000])

In [12]:
#topic_model.visualize_documents(docs[:100000], reduced_embeddings=reduced_embeddings, 
#                                hide_document_hover=True, hide_annotations=True)


In [13]:
new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings",embeddings=embeddings)
topic_model.update_topics(docs, topics=new_topics)


2024-08-29 05:42:24,665 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [14]:
df = topic_model.get_document_info(docs)

In [ ]:
df.to_csv('../data/topics_out.csv')

In [ ]:
topic_model.save('../model', serialization="safetensors", save_ctfidf=True)


In [ ]:
fig = topic_model.visualize_hierarchy()
fig.write_html("../data/hierarchy.html")